In [1]:
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(argparse))

Warning message:
“package ‘ggplot2’ was built under R version 4.2.3”


## Two Beta Model Vizualization

In [ ]:
# define command line arguments
parser <- ArgumentParser(description = "Visualize linear modeling results")
# add arguments
parser$add_argument('--celltype', type='character', help='Cell type to visualize')

# parse arguments from command line
args <- parser$parse_args()

# define cell type
celltype <- args$celltype


In [2]:

lm_file <- file.path(paste0("./results/", celltype, "/lm_two_beta.tsv"))

lm_cp_fig <- file.path(paste0("./figures/", celltype, "/lm_two_beta.pdf"))

# if path does not exist, create it
if (!dir.exists(file.path(paste0("./figures/", celltype)))) {
    dir.create(file.path(paste0("./figures/", celltype)))
}
lm_df <- readr::read_tsv(lm_file, col_types = readr::cols(.default = "d", feature ="c", inducer1_inhibitor_inhibitor_dose__inducer1_dose = "c"))
head(lm_df)

feature,r2_score,Metadata_number_of_singlecells,twob_Metadata_Treatment_Inhibitor_Dose,Treatment_Dose,inducer1_inhibitor_inhibitor_dose__inducer1_dose
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Cytoplasm_AreaShape_Area,9.343787e-04,1.695962e-05,0.034812093,-0.034812093,media ctr_Media ctr_0.0__0
Cytoplasm_AreaShape_BoundingBoxArea,1.740451e-03,-3.367326e-05,0.034966831,-0.034966831,media ctr_Media ctr_0.0__0
Cytoplasm_AreaShape_BoundingBoxMaximum_X,1.077223e-04,-2.358903e-05,-0.013234530,0.013234530,media ctr_Media ctr_0.0__0
Cytoplasm_AreaShape_BoundingBoxMaximum_Y,1.354607e-04,-2.231822e-05,0.004917124,-0.004917124,media ctr_Media ctr_0.0__0
Cytoplasm_AreaShape_BoundingBoxMinimum_X,1.515569e-04,-2.184418e-05,-0.015622436,0.015622436,media ctr_Media ctr_0.0__0
Cytoplasm_AreaShape_BoundingBoxMinimum_Y,8.637752e-05,-2.176965e-05,0.002139427,-0.002139427,media ctr_Media ctr_0.0__0


In [3]:
unique(lm_df$inducer1_inhibitor_inhibitor_dose__inducer1_dose)

[1] "media ctr_Media ctr_0.0__0"               
 [2] "DMSO_DMSO_1.0__0.100"                     
 [3] "DMSO_Z-VAD-FMK_100.0__0.100"              
 [4] "DMSO_Z-VAD-FMK_30.0__0.100"               
 [5] "DMSO_DMSO_0.025__0.100"                   
 [6] "Thapsigargin_DMSO_0.025__1.000"           
 [7] "Thapsigargin_DMSO_0.025__10.000"          
 [8] "Topotecan_DMSO_0.025__5.000"              
 [9] "Topotecan_DMSO_0.025__10.000"             
[10] "Topotecan_DMSO_0.025__20.000"             
[11] "LPS_DMSO_0.025__0.010"                    
[12] "LPS_DMSO_0.025__0.100"                    
[13] "LPS_DMSO_0.025__1.000"                    
[14] "LPS_DMSO_0.025__10.000"                   
[15] "LPS_Disulfiram_0.1__10.000"               
[16] "LPS_Disulfiram_1.0__10.000"               
[17] "LPS_Disulfiram_2.5__10.000"               
[18] "LPS_Nigericin_DMSO_0.025__100.000_1.0"    
[19] "LPS_Nigericin_DMSO_0.025__100.000_3.0"    
[20] "LPS_Nigericin_DMSO_0.025__100.000_10.0"   
[21] "Disulfiram_DMSO_0.025__0.100"             
[22] "Disulfiram_DMSO_0.025__1.000"             
[23] "Disulfiram_DMSO_0.025__2.500"             
[24] "H2O2_DMSO_0.025__100.000"                 
[25] "LPS_Z-VAD-FMK_100.0__10.000"              
[26] "LPS_DMSO_0.025__100.000"                  
[27] "LPS_Nigericin_DMSO_0.025__1.000_1.0"      
[28] "LPS_Nigericin_DMSO_0.025__1.000_3.0"      
[29] "LPS_Nigericin_DMSO_0.025__1.000_10.0"     
[30] "LPS_Nigericin_Disulfiram_1.0__1.000_10.0" 
[31] "LPS_Nigericin_Z-VAD-FMK_100.0__1.000_10.0"
[32] "H2O2_Disulfiram_1.0__100.000"             
[33] "H2O2_Z-VAD-FMK_100.0__100.000"            
[34] "Flagellin_DMSO_0.025__0.100"              
[35] "Flagellin_DMSO_0.025__1.000"              
[36] "Flagellin_Disulfiram_1.0__1.000"

In [4]:

# Arrange by absolute value coefficient
# Split out components of feature name for visualization
lm_df <- lm_df %>%
    dplyr::arrange(desc(abs(twob_Metadata_Treatment_Inhibitor_Dose))) %>%
    tidyr::separate(
        feature,
        into = c(
            "compartment",
            "feature_group",
            "measurement",
            "channel",
            "parameter1",
            "parameter2"
        ),
        sep = "_",
        remove = FALSE
    ) %>%
    dplyr::mutate(channel_cleaned = channel)



Warning message:
“Expected 6 pieces. Additional pieces discarded in 28080 rows [14, 15, 19, 21, 22, 23, 24, 25, 28, 31, 33, 34, 36, 37, 38, 42, 43, 44, 45, 46,
...].”
Warning message:
“Expected 6 pieces. Missing pieces filled with `NA` in 42012 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 20, 26, 27, 29, ...].”


In [5]:
unique(lm_df$channel)

[1] "CorrPM"        "CorrDNA"       "CorrER"        "CorrMito"     
 [5] "CorrGasdermin" NA              "Adjacent"      "Number"       
 [9] "8"             "0"             "4"             "6"            
[13] "5"             "3"             "1"             "7"            
[17] "2"             "9"             "X"             "Y"            
[21] "Z"             "Count"

In [6]:
unique(lm_df$channel_cleaned)

[1] "CorrPM"        "CorrDNA"       "CorrER"        "CorrMito"     
 [5] "CorrGasdermin" NA              "Adjacent"      "Number"       
 [9] "8"             "0"             "4"             "6"            
[13] "5"             "3"             "1"             "7"            
[17] "2"             "9"             "X"             "Y"            
[21] "Z"             "Count"

In [7]:
# Clean channel for visualization
lm_df$channel_learned <- dplyr::recode(lm_df$channel_cleaned,
        "CorrDNA" = "nuclei",
        "CorrMito" = "Mito",
        "CorrER" = "ER",
        "CorrGasdermin" = "gasdermin",
        "CorrPM" = "PM",
        .default = "other",
        .missing="other"
    )

print(dim(lm_df))
head(lm_df)
unique(lm_df$channel_learned)
lm_df$abs_Metadata_number_of_singlecells <- abs(lm_df$Metadata_number_of_singlecells)
lm_df$abs_twob_Metadata_Treatment_Inhibitor_Dose <- abs(lm_df$twob_Metadata_Treatment_Inhibitor_Dose)
lm_df$abs_Treatment_Dose <- abs(lm_df$Treatment_Dose)

[1] 102492     14


feature,compartment,feature_group,measurement,channel,parameter1,parameter2,r2_score,Metadata_number_of_singlecells,twob_Metadata_Treatment_Inhibitor_Dose,Treatment_Dose,inducer1_inhibitor_inhibitor_dose__inducer1_dose,channel_cleaned,channel_learned
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
Cytoplasm_Correlation_Manders_CorrPM_CorrMito,Cytoplasm,Correlation,Manders,CorrPM,CorrMito,NA,0.3974507,-1.330689e-05,-1.386229,-1.386229,H2O2_Disulfiram_1.0__100.000,CorrPM,PM
Cells_Correlation_Manders_CorrPM_CorrMito,Cells,Correlation,Manders,CorrPM,CorrMito,NA,0.3931833,-4.000428e-05,-1.368789,-1.368789,H2O2_Disulfiram_1.0__100.000,CorrPM,PM
Cells_Correlation_Manders_CorrDNA_CorrMito,Cells,Correlation,Manders,CorrDNA,CorrMito,NA,0.2683756,-6.649527e-05,-1.344787,-1.344787,H2O2_Disulfiram_1.0__100.000,CorrDNA,nuclei
Nuclei_Correlation_Manders_CorrPM_CorrMito,Nuclei,Correlation,Manders,CorrPM,CorrMito,NA,0.1566935,2.525405e-05,-1.323291,-1.323291,H2O2_Disulfiram_1.0__100.000,CorrPM,PM
Cytoplasm_Correlation_Manders_CorrDNA_CorrMito,Cytoplasm,Correlation,Manders,CorrDNA,CorrMito,NA,0.2701912,5.697376e-06,-1.267118,-1.267118,H2O2_Disulfiram_1.0__100.000,CorrDNA,nuclei
Nuclei_Correlation_Manders_CorrDNA_CorrMito,Nuclei,Correlation,Manders,CorrDNA,CorrMito,NA,0.1397890,5.213628e-05,-1.249437,-1.249437,H2O2_Disulfiram_1.0__100.000,CorrDNA,nuclei


[1] "PM"        "nuclei"    "ER"        "Mito"      "gasdermin" "other"

In [8]:
head(lm_df)

feature,compartment,feature_group,measurement,channel,parameter1,parameter2,r2_score,Metadata_number_of_singlecells,twob_Metadata_Treatment_Inhibitor_Dose,Treatment_Dose,inducer1_inhibitor_inhibitor_dose__inducer1_dose,channel_cleaned,channel_learned,abs_Metadata_number_of_singlecells,abs_twob_Metadata_Treatment_Inhibitor_Dose,abs_Treatment_Dose
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
Cytoplasm_Correlation_Manders_CorrPM_CorrMito,Cytoplasm,Correlation,Manders,CorrPM,CorrMito,NA,0.3974507,-1.330689e-05,-1.386229,-1.386229,H2O2_Disulfiram_1.0__100.000,CorrPM,PM,1.330689e-05,1.386229,1.386229
Cells_Correlation_Manders_CorrPM_CorrMito,Cells,Correlation,Manders,CorrPM,CorrMito,NA,0.3931833,-4.000428e-05,-1.368789,-1.368789,H2O2_Disulfiram_1.0__100.000,CorrPM,PM,4.000428e-05,1.368789,1.368789
Cells_Correlation_Manders_CorrDNA_CorrMito,Cells,Correlation,Manders,CorrDNA,CorrMito,NA,0.2683756,-6.649527e-05,-1.344787,-1.344787,H2O2_Disulfiram_1.0__100.000,CorrDNA,nuclei,6.649527e-05,1.344787,1.344787
Nuclei_Correlation_Manders_CorrPM_CorrMito,Nuclei,Correlation,Manders,CorrPM,CorrMito,NA,0.1566935,2.525405e-05,-1.323291,-1.323291,H2O2_Disulfiram_1.0__100.000,CorrPM,PM,2.525405e-05,1.323291,1.323291
Cytoplasm_Correlation_Manders_CorrDNA_CorrMito,Cytoplasm,Correlation,Manders,CorrDNA,CorrMito,NA,0.2701912,5.697376e-06,-1.267118,-1.267118,H2O2_Disulfiram_1.0__100.000,CorrDNA,nuclei,5.697376e-06,1.267118,1.267118
Nuclei_Correlation_Manders_CorrDNA_CorrMito,Nuclei,Correlation,Manders,CorrDNA,CorrMito,NA,0.1397890,5.213628e-05,-1.249437,-1.249437,H2O2_Disulfiram_1.0__100.000,CorrDNA,nuclei,5.213628e-05,1.249437,1.249437


In [9]:
loop_list <- unique(lm_df$inducer1_inhibitor_inhibitor_dose__inducer1_dose)
x_list <- c('abs_twob_Metadata_Treatment_Inhibitor_Dose','abs_Treatment_Dose')

In [10]:
pdf(file=lm_cp_fig )
for (i in 1:length(loop_list)){
    df <- lm_df[lm_df$inducer1_inhibitor_inhibitor_dose__inducer1_dose == loop_list[i],]
    for (j in 1:length(x_list)){
        lm_fig_gg <- (
            ggplot(df, aes(x = .data[[x_list[j]]], y = r2_score))
            + geom_point(aes(size = abs_Metadata_number_of_singlecells, color = channel_learned), alpha = 0.7)
            + theme_bw()
            + guides(
                color = guide_legend(title = "Channel\n(if applicable)", order = 1),
                size = guide_legend(title = "Cell count contributution")
            )
            + ylab("R2 score of LM feature")
            + xlab(paste0(x_list[j]," contribution (LM beta coefficient)"))
            + ggtitle(paste0("How CellProfiler features contribute\nto ",loop_list[i], "\ntreatments and cell density"))
        )
    plot(lm_fig_gg)    
    }
}
dev.off()

png 
  2